In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 40
import numpy as np
import os, shutil, time, re
from tqdm import tqdm

# Warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# Web Scraping
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from time import sleep
from bs4 import BeautifulSoup
from tqdm import tqdm
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-gpu")

options.add_argument("window-size=1920x1080")
options.add_argument("lang=ko_KR")
options.add_argument("user-agent=Chrome/77.0.3865.90")
download_path = os.getcwd()+'\\data\\search'
options.add_experimental_option("prefs", {
  "download.default_directory": download_path,
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
})

# 읍면동, 검색어
우도면, 추자면, 알수없음 제외

In [2]:
filepath = "./data/"
card = pd.read_csv(os.path.join(filepath,'04_음식관련 카드소비_CARD_SPENDING.csv'), encoding="cp949")
card = card.loc[~card.emd_nm.isin(['우도면','추자면'])].sort_values("base_date")
emd_nm_ = card.emd_nm.unique()
emd_nm_ = np.delete(emd_nm_, np.where(emd_nm_=="알수없음"))

emd_dict = dict(zip(list(emd_nm_),emd_nm_.reshape(-1,1).tolist()))
for k, v in emd_dict.items():
    if len(v[0])==3:
        v.append(v[0][:-1])
    if len(v[0])==4:
        v.append(v[0].replace('1','일').replace('2','이'))
    if len(v[0])==2:
        v.append("제주"+v[0])
emd_dict

{'삼양동': ['삼양동', '삼양'],
 '이호동': ['이호동', '이호'],
 '일도1동': ['일도1동', '일도일동'],
 '외도동': ['외도동', '외도'],
 '천지동': ['천지동', '천지'],
 '아라동': ['아라동', '아라'],
 '이도2동': ['이도2동', '이도이동'],
 '한경면': ['한경면', '한경'],
 '남원읍': ['남원읍', '남원'],
 '삼도2동': ['삼도2동', '삼도이동'],
 '일도2동': ['일도2동', '일도이동'],
 '용담1동': ['용담1동', '용담일동'],
 '화북동': ['화북동', '화북'],
 '애월읍': ['애월읍', '애월'],
 '중문동': ['중문동', '중문'],
 '용담2동': ['용담2동', '용담이동'],
 '영천동': ['영천동', '영천'],
 '오라동': ['오라동', '오라'],
 '이도1동': ['이도1동', '이도일동'],
 '송산동': ['송산동', '송산'],
 '대천동': ['대천동', '대천'],
 '성산읍': ['성산읍', '성산'],
 '대정읍': ['대정읍', '대정'],
 '서홍동': ['서홍동', '서홍'],
 '노형동': ['노형동', '노형'],
 '효돈동': ['효돈동', '효돈'],
 '구좌읍': ['구좌읍', '구좌'],
 '대륜동': ['대륜동', '대륜'],
 '봉개동': ['봉개동', '봉개'],
 '안덕면': ['안덕면', '안덕'],
 '표선면': ['표선면', '표선'],
 '예래동': ['예래동', '예래'],
 '연동': ['연동', '제주연동'],
 '동홍동': ['동홍동', '동홍'],
 '중앙동': ['중앙동', '중앙'],
 '조천읍': ['조천읍', '조천'],
 '한림읍': ['한림읍', '한림'],
 '도두동': ['도두동', '도두'],
 '건입동': ['건입동', '건입'],
 '삼도1동': ['삼도1동', '삼도일동'],
 '정방동': ['정방동', '정방']}

In [3]:
읍면동별_dic = {'읍면동별검색_생활' : '동사무소,우체국,보건소,주민센터,약국,병원,문화센터,도서관,이비인후과,내과,치과,산부인과,소아과,노래방,코인노래방,편의점',
                 '읍면동별검색_관광' : '여행,관광지,숙박,숙소,펜션,게스트하우스,호텔,에어비엔비,맛집,혼밥,카페,해변,해수욕장,골프장,날씨,가볼만한곳,코스,기념품,일몰,당일치기',
                 '읍면동별검색_쇼핑' : '다이소,홈플러스,이마트,백화점,아울렛,쇼핑몰,면세점,대형마트,마트,쇼핑'
                }

검색_dic = {'검색_관광' :'제주여행,제주맛집,제주펜션,제주술집,제주에어비앤비,제주가볼만한곳,제주항공,제주에어,제주비행기,제주항공가격,제주날씨,제주드라이브,제주일몰,제주면세점,제주롯데면세점,제주액티비티,제주렌트카,제주2박3일일정',
             '검색_관광2' : '제주도여행,제주도맛집,제주도펜션,제주도술집,제주도에어비앤비,제주도가볼만한곳,제주도항공,제주도에어,제주도비행기,제주도항공가격,제주도날씨,제주도드라이브,제주도일몰,제주도면세점,제주도롯데면세점,제주도액티비티,제주도렌트카,제주도2박3일일정',
             '검색_쇼핑' : '제주다이소,제주홈플러스,제주이마트,제주백화점,제주아울렛,제주쇼핑몰,제주면세점,제주대형마트,제주마트,제주쇼핑,제주도다이소,제주도홈플러스,제주도이마트,제주도백화점,제주도아울렛,제주도쇼핑몰,제주도면세점,제주도대형마트,제주도마트,제주도쇼핑',
             '검색_쓰레기' : '제주쓰레기,제주음식물쓰레기,제주매립장,제주일회용품,제주생활쓰레기,제주종량제,제주분리수거,제주클린하우스,제주재활용,제주플라스틱,제주폐기물',
             '검색_쓰레기2' : '제주도쓰레기,제주도음식물쓰레기,제주도매립장,제주도일회용품,제주도생활쓰레기,제주도종량제,제주도분리수거,제주도클린하우스,제주도재활용,제주도플라스틱,제주도폐기물',
             '검색_한파' : '제주폭설,제주강풍,제주한파,제주도폭설,제주도강풍,제주도한파',
             '검색_강풍' : '제주도강풍,제주강풍',
             '검색_태풍' : '제주도태풍,제주도폭풍,제주도해일,제주태풍,제주폭풍,제주해일,솔릭,콩레이,타파,폭우,마이삭,루핏,오마이스',
             '검색_장마' : '제주장마,제주호우주의보,제주호우주의,제주비,제주도장마,제주도호우주의보,제주도호우주의,제주도비',
             '검색_폭염' : '제주폭염, 제주더위, 제주열대야, 제주기온,제주도폭염, 제주도더위, 제주도열대야, 제주도기온',
             '검색_우박' : '제주우박,제주도우박',
             '검색_날씨' : '제주날씨, 제주도날씨',
             '검색_미세먼지' : '제주미세먼지, 제주미세미세, 제주도미세먼지, 제주도미세미세',
             '검색_황사' : '제주황사,제주도황사'}

# 웹스크래핑
2018-01-01 ~ 2021-08-31

In [4]:
def date_split(date):
    s, m, d = date.split("-")
    s = str(int(s)-2015)
    return s, m, d

def select_date(driver, d_name, d):
    sample = driver.find_element_by_xpath(f'//*[@id="{d_name}Div"]/ul/li[{d}]/a')
    driver.execute_script("arguments[0].click();", sample)

def download_xlsx(driver, url, colname, keyword, start, end):
    driver.get(url)
    # 주제어 입력
    driver.find_element_by_css_selector('input#item_keyword1').send_keys(colname) # column명 입력
    driver.find_element_by_css_selector('input#item_sub_keyword1_1').send_keys(keyword) # 검색 키워드 입력
    
    # 검색 기간 입력
    driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[3]/div/div/form/fieldset/div/div[6]/div[1]/label[5]').click() # 직접입력
    s_dict = dict(zip(['startYear', 'startMonth', 'startDay'],list(date_split(start)))) # 시작 날짜
    for k, v in s_dict.items():
        select_date(driver, k, v)
    e_dict = dict(zip(['endYear', 'endMonth', 'endDay'],list(date_split(end)))) # 종료 날짜
    for k, v in e_dict.items():
        select_date(driver, k, v)
        
    # 검색
    driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[3]/div/div/form/fieldset/a').send_keys((Keys.ENTER))
    time.sleep(3)
    
    # 검색 결과 다운로드
    driver.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[1]/div/div/div/div/div/div[1]/div[4]/a').send_keys((Keys.ENTER))
    time.sleep(5)
    
def get_search_emd(url, data, keyword_, start, end, filename, filepath="./data", option="each",emd_dict=None):
    driver = webdriver.Chrome(executable_path = 'C:/Program Files/chromedriver/chromedriver.exe', options = options)
    df_res = pd.DataFrame(pd.date_range(start=start,end=end), columns=["base_date"])
    if option=="each":
        for k, v in emd_dict.items():
            print(f"{k} 검색 중...")
            df_date = pd.DataFrame(pd.date_range(start=start,end=end), columns=["base_date"])
            for emd in v:
                # 데이터랩 검색 웹 스크래핑
                keyword = list(map(lambda x: emd+x.replace(' ',''), keyword_.split(',')))
                download_xlsx(driver, url, emd, ','.join(keyword), start, end) 
                print(f"-> {emd} 검색 완료")

                # 검색 결과 다운로드 후 원본 파일 삭제
                df_key = pd.read_excel(os.path.join(filepath, "datalab.xlsx"), header = 6).iloc[:,1]
                os.remove(os.path.join(filepath,'datalab.xlsx'))

                # 검색 결과 병합
                df_date = pd.concat([df_date, df_key], axis=1)

            # 동일 Case는 sum해서 병합
            df_sum = pd.DataFrame(df_date.set_index('base_date').sum(axis=1),columns=[k]).reset_index()
            df_res = pd.merge(df_res, df_sum, on="base_date", how="inner")
            print(df_res.head())
            
    elif option=="all":
        df_date = pd.DataFrame(pd.date_range(start=start,end=end), columns=["base_date"])
        # 데이터랩 검색 웹 스크래핑
        download_xlsx(driver, url, "search", keyword_, start, end) 

        # 검색 결과 다운로드 후 원본 파일 삭제
        df_key = pd.read_excel(os.path.join(filepath, "datalab.xlsx"), header = 6).iloc[:,1]
        os.remove(os.path.join(filepath,'datalab.xlsx'))

        # 검색 결과 병합
        df_res = pd.concat([df_date, df_key], axis=1)
        print(df_res.head())    
    return df_res

## 1. 읍면동별 키워드 검색 결과 
각 읍면동 검색 Case별로 Sum <br>
ex) 애월읍<br>
Case1. 애월읍 / 애월읍여행, 애월읍관광지, ...<br>
Case2. 애월 / 애월여행, 애월관광지, ...<br>
-> 2개 검색 결과의 합계를 애월읍의 검색 결과로 저장

※ 제주애월읍여행, ... 과 애월읍여행, ... 의 검색량 차이가 없어 '제주'를 붙이지 X

In [5]:
url = "https://datalab.naver.com/"
start = "2018-01-01"
end = "2021-08-31"
filepath = "./data/search"
for i, keyword in 읍면동별_dic.items():
    print(i)
    filename = f"{i}.csv"
    search_emd = get_search_emd(url, card, keyword, start, end, filename, filepath, "each", emd_dict)
    search_emd.to_csv(os.path.join(filepath, filename),index=False)
#     search_emd = pd.read_csv(os.path.join(filepath, filename))

읍면동별검색_생활
삼양동 검색 중...
-> 삼양동 검색 완료
-> 삼양 검색 완료
   base_date     삼양동
0 2018-01-01 10.9244
1 2018-01-02 68.4288
2 2018-01-03 49.0815
3 2018-01-04 44.3619
4 2018-01-05 43.5216
이호동 검색 중...
-> 이호동 검색 완료
-> 이호 검색 완료
   base_date     삼양동     이호동
0 2018-01-01 10.9244  0.0000
1 2018-01-02 68.4288 59.3117
2 2018-01-03 49.0815 15.7895
3 2018-01-04 44.3619 28.9474
4 2018-01-05 43.5216 18.4211
일도1동 검색 중...
-> 일도1동 검색 완료
-> 일도일동 검색 완료
   base_date     삼양동     이호동   일도1동
0 2018-01-01 10.9244  0.0000 0.0000
1 2018-01-02 68.4288 59.3117 0.0000
2 2018-01-03 49.0815 15.7895 0.0000
3 2018-01-04 44.3619 28.9474 0.0000
4 2018-01-05 43.5216 18.4211 0.0000
외도동 검색 중...
-> 외도동 검색 완료
-> 외도 검색 완료
   base_date     삼양동     이호동   일도1동     외도동
0 2018-01-01 10.9244  0.0000 0.0000  0.0000
1 2018-01-02 68.4288 59.3117 0.0000 49.4624
2 2018-01-03 49.0815 15.7895 0.0000 59.6774
3 2018-01-04 44.3619 28.9474 0.0000 22.5806
4 2018-01-05 43.5216 18.4211 0.0000 32.7957
천지동 검색 중...
-> 천지동 검색 완료
-> 천지 검색 완료
   base_date     삼양동 

## 2. 제주 전체 키워드 검색 결과 
특정 검색어에 대해 제주, 제주도를 붙여 검색 

In [6]:
url = "https://datalab.naver.com/"
start = "2018-01-01"
end = "2021-08-31"
filepath = "./data/search"
filename = "검색.csv"
df_res = pd.DataFrame(pd.date_range(start=start,end=end), columns=["base_date"])
for i, keyword in 검색_dic.items():
    print(f"{i} 검색 중...")
    search_emd = get_search_emd(url, card, keyword, start, end, filename, filepath,"all")
    search_emd.columns = ['base_date',f"{i}"]
    print(f"-> {i} 검색 완료")
    df_res = pd.merge(df_res, search_emd, on="base_date", how="inner")
df_res.to_csv(os.path.join(filepath, filename),index=False)

검색_관광 검색 중...
   base_date  search
0 2018-01-01 16.7493
1 2018-01-02 26.7391
2 2018-01-03 29.9351
3 2018-01-04 48.0833
4 2018-01-05 53.3408
-> 검색_관광 검색 완료
검색_관광2 검색 중...
   base_date  search
0 2018-01-01 14.6336
1 2018-01-02 18.0877
2 2018-01-03 18.7940
3 2018-01-04 19.3819
4 2018-01-05 18.2352
-> 검색_관광2 검색 완료
검색_쇼핑 검색 중...
   base_date  search
0 2018-01-01 40.8471
1 2018-01-02 44.2722
2 2018-01-03 42.3967
3 2018-01-04 43.9905
4 2018-01-05 42.9867
-> 검색_쇼핑 검색 완료
검색_쓰레기 검색 중...
   base_date  search
0 2018-01-01 20.6049
1 2018-01-02 27.4102
2 2018-01-03 27.4102
3 2018-01-04 31.9471
4 2018-01-05 18.7145
-> 검색_쓰레기 검색 완료
검색_쓰레기2 검색 중...
   base_date  search
0 2018-01-01 15.4142
1 2018-01-02 29.6724
2 2018-01-03 23.5067
3 2018-01-04 26.3969
4 2018-01-05 20.8092
-> 검색_쓰레기2 검색 완료
검색_한파 검색 중...
   base_date  search
0 2018-01-01  0.0632
1 2018-01-02  0.1060
2 2018-01-03  0.1134
3 2018-01-04  0.0762
4 2018-01-05  0.1915
-> 검색_한파 검색 완료
검색_강풍 검색 중...
   base_date  search
0 2018-01-01  0.0000
1 2018